In [8]:
# 构造CIG

In [ ]:
# textrank
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import networkx as nx

def textrank(sentences):
    """
    Given input text, split sentences and calc text rank score.
    :param sentences: input sentence list
    :return: a dictionary of (sentence index, sentence score)
    """
    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)
    similarity_graph = normalized * normalized.T
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return dict(((i, scores[i]) for i, s in enumerate(sentences)))


In [ ]:
# Encoding local matching vector

In [ ]:
# 计算tf

def gen_tf(text):
    """
    Given a segmented string, return a dict of tf.
    """
    tokens = text.split()
    total = len(tokens)
    tf_dict = {}
    for w in tokens:
        tf_dict[w] = tf_dict.get(w, 0.0) + 1.0
    for k in tf_dict:
        tf_dict[k] /= total
    return tf_dict

def tf_cos_sim(text1, text2):
    tf1 = gen_tf(text1)
    tf2 = gen_tf(text2)
    return cosine_sim(tf1, tf2)


In [2]:
# 计算tf-idf
def gen_tfidf(text, idf_dict):
    """
    Given a segmented string and idf dict, return a dict of tfidf.
    """
    tokens = text.split()
    total = len(tokens)
    tfidf_dict = {}
    for w in tokens:
        tfidf_dict[w] = tfidf_dict.get(w, 0.0) + 1.0
    for k in tfidf_dict:
        tfidf_dict[k] *= idf_dict.get(k, 0.0) / total
    return tfidf_dict

def cosine_sim(a, b):
    if len(b) < len(a):
        a, b = b, a
    res = 0
    for key, a_value in a.items():
        res += a_value * b.get(key, 0)
    if res == 0:
        return 0
    try:
        res = res / (norm(list(a.values())) * norm(list(b.values())))
    except ZeroDivisionError:
        res = 0
    return res

def tfidf_cos_sim(text1, text2, idf_dict):
    tfidf1 = gen_tfidf(text1, idf_dict)
    tfidf2 = gen_tfidf(text2, idf_dict)
    return cosine_sim(tfidf1, tfidf2)


In [4]:
# jaccard
def jaccard_common_words(text1, text2):
    str1 = set(str(text1).split())
    str2 = set(str(text2).split())
    if len(str1) == 0 or len(str2) == 0:
        return 0.0
    return float(len(str1 & str2)) / len(str1 | str2)

In [5]:
# ochiai
def ochiai_common_words(text1, text2):
    str1 = set(str(text1).split())
    str2 = set(str(text2).split())
    if len(str1) == 0 or len(str2) == 0:
        return 0.0
    return float(len(str1 & str2)) / math.sqrt(len(str1) * len(str2))

In [7]:
# build_bm25
from gensim.summarization import bm25
def build_bm25(corpus):
    corpus = []
    bm25Model = bm25.BM25(corpus)

    return bm25Model


def cal_bm25_sim(bm25Model, tokens1, tokens2):
    ts1, ts2 = tokens1, tokens2
    if len(tokens1) > len(tokens2):
        ts1 = tokens2
        ts2 = tokens1
    freqs = {}
    for word in ts2:
        if word not in freqs:
            freqs[word] = 0
        freqs[word] += 1

    param_k1 = 1.5
    param_b = 0.75
    score1, score2 = 0.0, 0.0
    for word in ts1:
        if word not in freqs or word not in bm25Model.idf:
            continue
        score1 += (bm25Model.idf[word] * freqs[word] * (param_k1 + 1) / (
            freqs[word] + param_k1 * (1 - param_b + param_b * 1)))
    for word in ts2:
        if word not in freqs or word not in bm25Model.idf:
            continue
        score2 += (bm25Model.idf[word] * freqs[word] * (param_k1 + 1) / (
            freqs[word] + param_k1 * (1 - param_b + param_b * 1)))

    sim = score1 / score2 if score2 > 0 else 0
    sim = sim if sim <= 1.0 else 1.0
    return sim

tokens1 = '我们 的 目标 就 是 能够 使用 海量 用户 搜索 日志'
tokens2 = '在 海量 数据 里 挖掘 潜藏 的 查询 之间 的 结构 信息'
sim = cal_bm25_sim(bm25Model, tokens1, tokens2)